# Problema de los ascensores

#### G05 - Boris Carballa Corredoira, Juan Carlos Villanueva Quirós, Francisco Javier Blázquez Martínez

El edificio consta de varios bloques con varios ascensores.

In [70]:
from search import *
import copy

In [87]:
# CONSTANTES
NUM_PLANTAS    = 12 #Para rango por defecto del ascensor
NUM_ASCENSORES = 4
NUM_PERSONAS = 5 #Se usa en quita_dejar_coger_inutiles_acciones
CAP_ASCENSOR   = 2
EN_ASCENSOR = -1
NINGUNA = -1
EN_DESTINO = -2

In [88]:
%run SolutionManager.ipynb

In [89]:
# El estado de un nodo del espacio de exploración se compone de:
# ascensor_pos      -> Planta en la que se encuentra el ascensor
# ascensor_personas -> Personas dentro del ascensor
# personas          -> Lista con planta en la que se encuentra cada persona 
#                      (Pueden tomar el valor de la constante EN_ASCENSOR)
class NodeState:

    def __init__(self, init_personas, ascensor_rango=None):
        self.personas = init_personas
        self.ascensor_personas = [[] for _ in range(NUM_ASCENSORES)]
        if ascensor_rango is None:
            self.ascensor_rango = [(0,NUM_PLANTAS) for _ in range(NUM_ASCENSORES)] #[min,max], te deja lo mas arriba/abajo que puede
            self.ascensor_pos = [0 for _ in range(NUM_ASCENSORES)]
        else:
            self.ascensor_rango = ascensor_rango
            self.ascensor_pos = [rango[0] for rango in ascensor_rango]
        
    def __eq__(self, nodo):
        return self.personas == nodo.personas and self.ascensor_pos == nodo.ascensor_pos and self.ascensor_personas == nodo.ascensor_personas
   
    def __hash__(self):
        return hash((tuple(self.personas), tuple(self.ascensor_pos), tuple(tuple(x) for x in self.ascensor_personas)))
    
    def __lt__(self,nodo):
        return True

CAVEAT: ascensor_rango no deberia ser un atributo de NODE_STATE sino de PROBLEM -> GASTAMOS MAS MEMORIA

In [95]:
# Definimos la clase para la representación del problema. Las acciones
# contempladas son hacer al ascensor subir o bajar hasta la planta con
# personas (o planta destino de los ocupantes) más cercana y recoger o
# dejar a una persona en una planta.

class AscensoresSimplificado(Problem) :    
   
    def __init__(self, initial, goal=None):        
        self.initial = initial
        self.goal = goal
        self.analizados = 0

    # Codificación acciones:
    # (BAJAR, i, j) -> El ascensor i baja j plantas
    # (SUBIR, i, j) -> El ascensor i sube j plantas
    # (COGER_PERSONA, i, j) -> El ascensor i coge a la persona j
    # (DEJAR_PERSONA, i, j) -> El ascensor i deja a la persona j
    def actions(self, state):
        
        accs = list()
        
        for num_ascensor in range(0,NUM_ASCENSORES):
            ascensor_pos      = state.ascensor_pos[num_ascensor]
            ascensor_rango    = state.ascensor_rango[num_ascensor]
            ascensor_personas = state.ascensor_personas[num_ascensor]
            personas          = state.personas

            ascensor_vacio    = not ascensor_personas
            ascensor_lleno    = (len(ascensor_personas)==CAP_ASCENSOR)

            personas_esta_planta_suben = [i for i in range(len(personas)) if personas[i]==ascensor_pos and self.goal[i]>ascensor_pos]
            personas_esta_planta_bajan = [i for i in range(len(personas)) if personas[i]==ascensor_pos and self.goal[i]<ascensor_pos]


            # 1.- Si alguien ha llegado a su destino o no podemos ir más cerca, forzamos que baje del ascensor
            for persona in ascensor_personas:
                if self.goal[persona]==ascensor_pos or \
                (self.goal[persona]<ascensor_rango[0] and ascensor_pos==ascensor_rango[0]) or \
                (self.goal[persona]>ascensor_rango[1] and ascensor_pos==ascensor_rango[1]):
                    return [(DEJAR_PERSONA,num_ascensor,persona)]


            # 2.- Para la acción subir requerimos que haya que coger o dejar a alguien
            #     más arriba de donde se encuentra el ascensor. Nunca subimos con gente
            #     que desea bajar plantas y además, un ascensor no sube si tiene capacidad
            #     para más personas y en esa planta hay alguien que desea subir (eliminación
            #     de ramas sin perder optimaldad).

            persona_menos_arriba = min([person for person in personas if ascensor_pos<person and person<=ascensor_rango[1] and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)

            # Si el ascensor está vacío sólo subimos si hay alguna persona por arriba 
            # y no hay gente en esta planta que quiera subir
            if ascensor_vacio:
                if persona_menos_arriba!=NINGUNA and not(personas_esta_planta_suben):
                    accs.append((SUBIR,num_ascensor,persona_menos_arriba-ascensor_pos))

            # Si el ascensor no está vacío comprobamos que sus personas suben y vamos
            # al mínimo entre a donde suben estas y donde hay una persona por encima
            # siempre que no nos quede espacio o no haya gente en esa planta que quiere subir
            elif self.goal[ascensor_personas[0]]>ascensor_pos:
                destino_menos_arriba = min(ascensor_rango[1], min([self.goal[i] for i in ascensor_personas]))

                if ascensor_lleno or not(personas_esta_planta_suben):
                    if persona_menos_arriba==NINGUNA:
                        accs.append((SUBIR,num_ascensor,destino_menos_arriba-ascensor_pos))
                    else:
                        accs.append((SUBIR,num_ascensor,min(persona_menos_arriba, destino_menos_arriba)-ascensor_pos))


            # 3.- La acción bajar es totalmente dual a la acción subir, mismas lógica en
            #     las condiciones. 

            persona_menos_abajo = max([person for person in personas if ascensor_pos>person and person>=ascensor_rango[0] and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)

            # Si el ascensor está vacío sólo bajamos si hay alguna persona por abajo
            if ascensor_vacio:
                if persona_menos_abajo!=NINGUNA and not(personas_esta_planta_bajan):
                    accs.append((BAJAR,num_ascensor,ascensor_pos-persona_menos_abajo))
            elif self.goal[ascensor_personas[0]]<ascensor_pos:
                destino_menos_abajo = max(ascensor_rango[0], max([self.goal[i] for i in ascensor_personas]))

                if ascensor_lleno or not(personas_esta_planta_bajan):
                    if persona_menos_abajo==NINGUNA:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-destino_menos_abajo))
                    else:
                        accs.append((BAJAR,num_ascensor,ascensor_pos-max(persona_menos_abajo, destino_menos_abajo)))

            # 4.- Acción dejar persona:
            for personaInterior in ascensor_personas:
                accs.append((DEJAR_PERSONA,num_ascensor,personaInterior))

            # 5.- Acción coger persona. Sólo cogemos a personas que suben o bajan, no ambas
            if not ascensor_lleno:
                if ascensor_vacio:
                    if ascensor_pos != ascensor_rango[1]:
                        for persona in personas_esta_planta_suben:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                    if ascensor_pos != ascensor_rango[0]:
                        for persona in personas_esta_planta_bajan:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                else: #si tiene a alguien dentro, ese alguien va hacia adentro del rango, si no forzábamos su salida en 1.-
                    if self.goal[ascensor_personas[0]]<ascensor_pos:
                        for persona in personas_esta_planta_bajan:
                            accs.append((COGER_PERSONA,num_ascensor,persona))
                    else:
                        for persona in personas_esta_planta_suben:
                            accs.append((COGER_PERSONA,num_ascensor,persona))

        return accs

    def result(self, state, action):
        estado_nuevo = copy.deepcopy(state)
        
        ascensor = action[1]
        
        if action[0]==BAJAR:
            estado_nuevo.ascensor_pos[ascensor] -= action[2]
        elif action[0]==SUBIR:
            estado_nuevo.ascensor_pos[ascensor] += action[2]
        elif action[0]==COGER_PERSONA:
            estado_nuevo.ascensor_personas[ascensor].append(action[2])
            estado_nuevo.personas[action[2]] = EN_ASCENSOR
        else:
            estado_nuevo.ascensor_personas[ascensor].remove(action[2])
            
            if self.goal[action[2]]==state.ascensor_pos[ascensor]:
                estado_nuevo.personas[action[2]] = EN_DESTINO
            else:
                estado_nuevo.personas[action[2]] = state.ascensor_pos[ascensor]
        
        return estado_nuevo
    
    def goal_test(self, state):
        self.analizados +=1
        return state.personas == [EN_DESTINO for _ in state.personas]

    def path_cost(self, c, state1, action, state2):
        if action[0]==BAJAR or action[0]==SUBIR:
            return c + action[2]
        else:
            return c
    
    def h(self,node):
        suma = sum([abs(self.goal[i]-node.state.personas[i]) for i in range(0,len(node.state.personas)) if node.state.personas[i]!=EN_DESTINO and node.state.personas[i]!=EN_ASCENSOR])
        for ascensor in range(0, len(node.state.ascensor_personas)):
            suma += sum([abs(self.goal[persona]-node.state.ascensor_pos[ascensor]) for persona in node.state.ascensor_personas[ascensor]])
        return suma
        
    def value(self, state):
        raise NotImplementedError

In [96]:
# Para probar, hay que cambiar los parámetros de arriba también, no solo init
init = NodeState([2,4,1,8,1],[(0,4),(4,8),(4,8),(8,12)])
goal = [3,11,12,1,9]
problem = AscensoresSimplificado(init, goal)

In [97]:
problem.analizados=0
answer = astar_search(problem)
if answer is None:
    acciones = []
    print("No hay solución")
else:
    acciones = cocina_acciones(answer.solution())
    decodificador_acciones(acciones)

El ascensor 1 coge a la persona 1
El ascensor 1 sube 4 plantas
El ascensor 1 deja a la persona 1
El ascensor 0 sube 1 plantas
El ascensor 0 coge a la persona 4
El ascensor 3 coge a la persona 1
El ascensor 3 sube 3 plantas
El ascensor 3 deja a la persona 1
El ascensor 1 coge a la persona 3
El ascensor 1 baja 4 plantas
El ascensor 1 deja a la persona 3
El ascensor 0 coge a la persona 2
El ascensor 0 sube 3 plantas
El ascensor 0 deja a la persona 4
El ascensor 0 deja a la persona 2
El ascensor 2 coge a la persona 4
El ascensor 2 coge a la persona 2
El ascensor 2 sube 4 plantas
El ascensor 2 deja a la persona 4
El ascensor 2 deja a la persona 2
El ascensor 0 coge a la persona 3
El ascensor 0 baja 3 plantas
El ascensor 0 deja a la persona 3
El ascensor 3 baja 3 plantas
El ascensor 3 coge a la persona 4
El ascensor 3 coge a la persona 2
El ascensor 3 sube 1 plantas
El ascensor 3 deja a la persona 4
El ascensor 3 sube 3 plantas
El ascensor 3 deja a la persona 2
El ascensor 0 sube 1 plantas
E

In [98]:
problem.analizados

77

In [99]:
coste_acciones(acciones)

31